In [70]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import warnings
import time
from collections import Counter
warnings.filterwarnings("ignore")


plt.style.use("fivethirtyeight")

sns.set_style({'font.sans-serif': ['simsun', 'Arial']})
sns.set_style('darkgrid', {'font.sans-serif': ['simhei', 'Arial']})
%matplotlib inline
current_date = pd.to_datetime(
    time.strftime('%Y-%m-%d', time.localtime(time.time())))

res = {}
data_path = r'./Documents/赛题1数据集/'
current_date

Timestamp('2019-04-02 00:00:00')

# 资产负债表

In [71]:
df_asset_bpk = pd.read_excel(data_path+r'上市信息财务信息资产负债表.xlsx')
df_asset = df_asset_bpk.copy()

In [72]:
df_asset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19620 entries, 0 to 19619
Data columns (total 24 columns):
企业编号               19620 non-null int64
标题                 19620 non-null object
日期                 19620 non-null object
资产:货币资金(元)         18862 non-null object
资产:应收账款(元)         18291 non-null object
资产:其它应收款(元)        18460 non-null object
资产:存货(元)           18115 non-null object
资产:流动资产合计(元)       18455 non-null object
资产:长期股权投资(元)       12873 non-null object
资产:累计折旧(元)         1446 non-null object
资产:固定资产(元)         18854 non-null object
资产:无形资产(元)         18595 non-null object
资产:资产总计(元)         18862 non-null object
负债:应付账款(元)         15646 non-null object
负债:预收账款(元)         18117 non-null object
负债:存货跌价准备(元)       1446 non-null object
负债:流动负债合计(元)       18455 non-null object
负债:长期负债合计(元)       17729 non-null object
负债:负债合计(元)         18862 non-null object
权益:实收资本(或股本)(元)    18851 non-null object
权益:资本公积金(元)        18731 non-null object
权益:盈余公积金(元)        18669 non-null

In [73]:
A = ['资产:货币资金(元)', '资产:长期股权投资(元)', '资产:固定资产(元)', '资产:无形资产(元)','负债:长期负债合计(元)','权益:实收资本(或股本)(元)',
     '资产:流动资产合计(元)','资产:资产总计(元)','负债:流动负债合计(元)','负债:负债合计(元)','权益:股东权益合计(元)']
df_asset=df_asset[~df_asset['权益:股东权益合计(元)'].isin(['正无穷大万亿'])]
df_asset1= pd.DataFrame({'企业编号':df_asset['企业编号'],'资产:货币资金(元)':df_asset['资产:货币资金(元)'],'资产:长期股权投资(元)': df_asset['资产:长期股权投资(元)'],
                         '资产:固定资产(元)': df_asset['资产:固定资产(元)'],'资产:无形资产(元)': df_asset['资产:无形资产(元)'],'负债:长期负债合计(元)':df_asset['负债:长期负债合计(元)'],
                         '权益:实收资本(或股本)(元)':df_asset['权益:实收资本(或股本)(元)'],'资产:流动资产合计(元)': df_asset['资产:流动资产合计(元)'],
                         '资产:资产总计(元)': df_asset['资产:资产总计(元)'],'负债:流动负债合计(元)': df_asset['负债:流动负债合计(元)'],'负债:负债合计(元)':df_asset['负债:负债合计(元)'],
                         '权益:股东权益合计(元)':df_asset['权益:股东权益合计(元)'],
})
df_asset1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 19619 entries, 0 to 19619
Data columns (total 12 columns):
企业编号               19619 non-null int64
资产:货币资金(元)         18861 non-null object
资产:长期股权投资(元)       12872 non-null object
资产:固定资产(元)         18853 non-null object
资产:无形资产(元)         18594 non-null object
负债:长期负债合计(元)       17728 non-null object
权益:实收资本(或股本)(元)    18850 non-null object
资产:流动资产合计(元)       18454 non-null object
资产:资产总计(元)         18861 non-null object
负债:流动负债合计(元)       18454 non-null object
负债:负债合计(元)         18861 non-null object
权益:股东权益合计(元)       18853 non-null object
dtypes: int64(1), object(11)
memory usage: 1.9+ MB


In [74]:
for arr in A:
    chinese = set(df_asset1[arr].str.extract(
        "([^0-9\.\-]+)", expand=False).dropna())
    print(arr, chinese)
for arr in A:
    df_asset1.loc[(df_asset1[arr].str.contains("万") == True), arr] = df_asset1.loc[(
        df_asset1[arr].str.contains("万") == True), arr].str.replace('万', '').astype(float)*0.0001
    df_asset1.loc[(df_asset1[arr].str.contains("亿") == True), arr] = df_asset1.loc[(
        df_asset1[arr].str.contains("亿") == True), arr].str.replace('亿', '').astype(float)*1
for arr in A:
     df_asset1[arr]=pd.to_numeric( df_asset1[arr],errors='coerce')

资产:货币资金(元) {'万', '亿'}
资产:长期股权投资(元) {'万', '亿'}
资产:固定资产(元) {'万', '亿'}
资产:无形资产(元) {'万', '亿'}
负债:长期负债合计(元) {'万', '亿'}
权益:实收资本(或股本)(元) {'万', '亿'}
资产:流动资产合计(元) {'万', '亿'}
资产:资产总计(元) {'万', '亿'}
负债:流动负债合计(元) {'万', '亿'}
负债:负债合计(元) {'万', '亿'}
权益:股东权益合计(元) {'万', '亿'}


In [76]:
df_asset1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19619 entries, 0 to 19619
Data columns (total 12 columns):
企业编号               19619 non-null int64
资产:货币资金(元)         18831 non-null float64
资产:长期股权投资(元)       12342 non-null float64
资产:固定资产(元)         18823 non-null float64
资产:无形资产(元)         18540 non-null float64
负债:长期负债合计(元)       17643 non-null float64
权益:实收资本(或股本)(元)    18820 non-null float64
资产:流动资产合计(元)       18424 non-null float64
资产:资产总计(元)         18835 non-null float64
负债:流动负债合计(元)       18424 non-null float64
负债:负债合计(元)         18831 non-null float64
权益:股东权益合计(元)       18823 non-null float64
dtypes: float64(11), int64(1)
memory usage: 1.9 MB


In [77]:
df_asset1.describe()


,企业编号,资产:货币资金(元),资产:长期股权投资(元),资产:固定资产(元),资产:无形资产(元),负债:长期负债合计(元),权益:实收资本(或股本)(元),资产:流动资产合计(元),资产:资产总计(元),负债:流动负债合计(元),负债:负债合计(元),权益:股东权益合计(元)
count,19619.000000,18831.000000,12342.000000,1.882300e+04,1.854000e+04,1.764300e+04,18820.000000,18424.000000,18835.000000,18424.000000,18831.000000,18823.000000
mean,2502.105102,142.584761,363.644876,4.029315e+02,5.467092e+03,1.008947e+04,16.540032,46.438059,43.027878,50.199079,50.217094,46.735165
std,861.625393,6427.783670,5438.401654,2.290755e+04,6.473206e+05,8.152211e+05,74.559939,713.716339,634.801261,566.068392,549.517782,1073.566228
min,1001.000000,-99.944766,-427.729651,-1.000000e+02,-9.997863e+01,-1.502334e+02,-72.556880,-98.036005,-82.513013,-97.254836,-97.063776,-14251.488710
25%,1756.000000,-19.485657,-2.845502,-4.576672e+00,-3.822040e+00,-1.591428e+01,0.000000,0.165046,2.501147,-2.596546,-2.020631,2.499618
50%,2514.000000,4.950000,5.756586,2.698150e+00,1.098947e-01,1.810000e+00,0.000000,11.959687,11.078739,13.758170,11.807447,7.905877
75%,3240.500000,42.707068,41.036065,1.853456e+01,1.454140e+01,4.660194e+01,10.214753,31.492096,27.314823,40.725344,37.818915,21.096949
max,4000.000000,622866.080393,381473.529340,1.948040e+06,8.790694e+07,1.047902e+08,4939.714566,45279.835862,43345.620140,44971.460810,43868.417546,85332.932385


In [117]:
res['资产负债表']=df_asset1

# 利润表

In [157]:
df_profit_bpk = pd.read_excel(data_path+r'上市信息财务信息-利润表.xlsx')
df_profit = df_profit_bpk.copy()
B=['营业成本(元)','销售费用(元)','财务费用(元)','管理费用(元)','投资收益(元)','营业利润(元)','利润总额(元)','归属母公司所有者净利润(元)']
for arr in B:
    chinese = set(df_profit[arr].str.extract(
        "([^0-9\.\-]+)", expand=False).dropna())
    print(arr, chinese)
for arr in B:
    df_profit.loc[(df_profit[arr].str.contains("万") == True), arr] = df_profit.loc[(
       df_profit[arr].str.contains("万") == True), arr].str.replace('万', '').astype(float)*10000
    df_profit.loc[(df_profit[arr].str.contains("亿") == True), arr] = df_profit.loc[(
        df_profit[arr].str.contains("亿") == True), arr].str.replace('亿', '').astype(float)*100000000
    df_profit[arr]=pd.to_numeric(df_profit[arr],errors='coerce')
df_profit1=df_profit.drop(['标题','日期','营业收入(元)','资产减值损失(元)','资产减值损失(元)','所得税(元)'],axis=1)
df_profit1.describe()
df_profit1.info()

营业成本(元) {'万', '亿'}
销售费用(元) {'万', '亿'}
财务费用(元) {'万', '亿'}
管理费用(元) {'万', '亿'}
投资收益(元) {'万', '亿'}
营业利润(元) {'万', '亿'}
利润总额(元) {'万', '亿'}
归属母公司所有者净利润(元) {'万', '亿'}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19621 entries, 0 to 19620
Data columns (total 9 columns):
企业编号              19621 non-null int64
营业成本(元)           19174 non-null float64
销售费用(元)           18717 non-null float64
财务费用(元)           19197 non-null float64
管理费用(元)           19621 non-null float64
投资收益(元)           17532 non-null float64
营业利润(元)           19613 non-null float64
利润总额(元)           19613 non-null float64
归属母公司所有者净利润(元)    19600 non-null float64
dtypes: float64(8), int64(1)
memory usage: 1.3 MB


In [158]:
res['利润表']=df_profit1

# 现金流量表

In [105]:
df_cash_bpk = pd.read_excel(data_path+r'上市信息财务信息-现金流量表.xlsx')
df_cash = df_cash_bpk.copy()

In [106]:
df_cash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19621 entries, 0 to 19620
Data columns (total 27 columns):
企业编号                               19621 non-null int64
标题                                 19621 non-null object
日期                                 19621 non-null object
经营:销售商品、提供劳务收到的现金(元)               19191 non-null object
经营:收到的税费返还(元)                      13718 non-null object
经营:收到其他与经营活动有关的现金(元)               19602 non-null object
经营:经营活动现金流入小计(元)                   19621 non-null object
经营:购买商品、接受劳务支付的现金(元)               19160 non-null object
经营:支付给职工以及为职工支付的现金(元)              19620 non-null object
经营:支付的各项税费(元)                      19620 non-null object
经营:支付其他与经营活动有关的现金(元)               19620 non-null object
经营:经营活动现金流出小计(元)                   19621 non-null object
经营:经营活动产生的现金流量净额(元)                19621 non-null object
投资:取得投资收益所收到的现金(元)                 14283 non-null object
投资:处置固定资产、无形资产和其他长期资产收回的现金净额(元)    16016 non-null object
投资:投资活动现金流入小计(元)                   18

In [115]:
df_cash1=pd.DataFrame({'企业编号':df_cash['企业编号'],'经营流入':df_cash['经营:经营活动现金流入小计(元)'],
                '经营净额':df_cash['经营:经营活动产生的现金流量净额(元)'], '投资流出':df_cash['投资:投资活动现金流出小计(元)'], '投资净额':df_cash['投资:投资活动产生的现金流量净额(元)'],
                  '筹资流入':df_cash['筹资:筹资活动现金流入小计(元)'],'筹资净额':df_cash['筹资活动产生的现金流量净额(元)'],})
C=['经营流入','经营净额','投资流出','投资净额', '筹资流入','筹资净额']
for arr in C:
    chinese = set(df_cash1[arr].str.extract(
        "([^0-9\.\-]+)", expand=False).dropna())
    print(arr, chinese)
for arr in C:
    df_cash1.loc[(df_cash1[arr].str.contains("万") == True), arr] = df_cash1.loc[(
        df_cash1[arr].str.contains("万") == True), arr].str.replace('万', '').astype(float)*0.0001
    df_cash1.loc[(df_cash1[arr].str.contains("亿") == True), arr] = df_cash1.loc[(
        df_cash1[arr].str.contains("亿") == True), arr].str.replace('亿', '').astype(float)*1
    df_cash1[arr]=pd.to_numeric(df_cash1[arr],errors='coerce')
df_cash1.info()

经营流入 {'万', '亿'}
经营净额 {'万', '亿'}
投资流出 {'万', '亿'}
投资净额 {'万', '亿'}
筹资流入 {'万', '亿'}
筹资净额 {'万', '亿'}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19621 entries, 0 to 19620
Data columns (total 7 columns):
企业编号    19621 non-null int64
经营流入    19621 non-null float64
经营净额    19621 non-null float64
投资流出    19550 non-null float64
投资净额    19566 non-null float64
筹资流入    17268 non-null float64
筹资净额    19066 non-null float64
dtypes: float64(6), int64(1)
memory usage: 1.0 MB


In [119]:
res['现金流量表']=df_cash1

# 每股指标

In [128]:
df_meiguzhibiao_bpk = pd.read_excel(data_path+r'上市公司财务信息-每股指标.xlsx')
df_meiguzhibiao = df_meiguzhibiao_bpk.copy()
df_meiguzhibiao1=df_meiguzhibiao.drop(['标题','日期','稀释每股收益(元)'],axis=1)
D=['基本每股收益(元)','扣非每股收益(元)','每股净资产(元)','每股公积金(元)','每股未分配利润(元)','每股经营现金流(元)']
for arr in D:
    df_meiguzhibiao1[arr]=pd.to_numeric(df_meiguzhibiao1[arr],errors='coerce')
df_meiguzhibiao1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28887 entries, 0 to 28886
Data columns (total 7 columns):
企业编号          28887 non-null int64
基本每股收益(元)     28824 non-null float64
扣非每股收益(元)     28073 non-null float64
每股净资产(元)      25597 non-null float64
每股公积金(元)      25456 non-null float64
每股未分配利润(元)    25586 non-null float64
每股经营现金流(元)    25613 non-null float64
dtypes: float64(6), int64(1)
memory usage: 1.5 MB


In [129]:
res['每股指标']=df_meiguzhibiao1

# 成长能力指标

In [134]:
df_growth_bpk = pd.read_excel(data_path+r'上市信息财务信息-成长能力指标.xlsx')
df_growth= df_growth_bpk.copy()
df_growth=df_growth.drop(['标题','日期','扣非净利润(元)','归属净利润同比增长(元)','营业总收入滚动环比增长(元)', '归属净利润滚动环比增长(元)', '扣非净利润滚动环比增长(元)'],axis=1)
E = ['营业总收入(元)', '毛利润(元)', '归属净利润(元)']
for arr in E:
    df_growth.loc[(df_growth[arr].str.contains("万亿") == True), arr] = df_growth.loc[(
        df_growth[arr].str.contains("万亿") == True), arr].str.replace('万亿', '').astype(float)*1000000000000
    df_growth.loc[(df_growth[arr].str.contains("万") == True), arr] = df_growth.loc[(
        df_growth[arr].str.contains("万") == True), arr].str.replace('万', '').astype(float)*10000
    df_growth.loc[(df_growth[arr].str.contains("亿") == True), arr] = df_growth.loc[(
        df_growth[arr].str.contains("亿") == True), arr].str.replace('亿', '').astype(float)*100000000
F = ['营业总收入同比增长(元)', '扣非净利润同比增长(元)']
for arr in F:
    df_growth[arr] = df_growth[arr].replace('--%', np.nan)
for arr in F:
    df_growth.loc[(df_growth[arr].str.contains("%") == True), arr] = df_growth.loc[(
        df_growth[arr].str.contains("%") == True), arr].str.replace(',', '').str.rstrip('%').astype(float)/100
df_growth = df_growth.convert_objects(convert_numeric=True)

In [135]:
df_growth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28887 entries, 0 to 28886
Data columns (total 6 columns):
企业编号            28887 non-null int64
营业总收入(元)        28877 non-null float64
毛利润(元)          28185 non-null float64
归属净利润(元)        28887 non-null float64
营业总收入同比增长(元)    28229 non-null float64
扣非净利润同比增长(元)    22907 non-null float64
dtypes: float64(5), int64(1)
memory usage: 1.3 MB


In [136]:
res['成长能力']=df_growth

# 盈利能力指标

In [148]:
df_abilityofprofit_bpk= pd.read_excel(data_path+r'上市信息财务信息盈利能力指标.xlsx')
df_abilityofprofit= df_abilityofprofit_bpk.copy()
G=['加权净资产收益率(%)','摊薄净资产收益率(%)','摊薄总资产收益率(%)','毛利率(%)','净利率(%)','实际税率(%)']
for arr in G:
        df_abilityofprofit[arr] = df_abilityofprofit[arr].replace('--%', np.nan)
        df_abilityofprofit[arr] =df_abilityofprofit[arr].str.rstrip('%').astype(float)/100
df_abilityofprofit.convert_objects(convert_numeric=True)
df_abilityofprofit=df_abilityofprofit.drop(['标题','日期'],axis=1)
df_abilityofprofit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28887 entries, 0 to 28886
Data columns (total 7 columns):
企业编号           28887 non-null int64
加权净资产收益率(%)    28218 non-null float64
摊薄净资产收益率(%)    28316 non-null float64
摊薄总资产收益率(%)    28887 non-null float64
毛利率(%)         28203 non-null float64
净利率(%)         28875 non-null float64
实际税率(%)        25648 non-null float64
dtypes: float64(6), int64(1)
memory usage: 1.5 MB


In [142]:
res['盈利能力能力']=df_abilityofprofit

# 财务风险指标

In [154]:
df_risk_bpk= pd.read_excel(data_path+r'上市信息财务信息-财务风险指标.xlsx')
df_risk= df_risk_bpk.copy()
H=['流动比率','速动比率']
for arr in H:
      df_risk[arr] = pd.to_numeric(df_risk[arr],errors='coerce')
I=['资产负债率(%)','流动负债/总负债(%)']
for arr in I:
     df_risk[arr] = df_risk[arr].replace('--%', np.nan)
df_risk.convert_objects(convert_numeric=True)
for arr in I:
    df_risk[arr] =df_risk[arr].str.rstrip('%').astype(float)/100
df_risk.convert_objects(convert_numeric=True)
df_risk=df_risk.drop(['标题','日期','速动比率'],axis=1)
df_risk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28887 entries, 0 to 28886
Data columns (total 4 columns):
企业编号           28887 non-null int64
资产负债率(%)       28437 non-null float64
流动负债/总负债(%)    28355 non-null float64
流动比率           27703 non-null float64
dtypes: float64(3), int64(1)
memory usage: 902.8 KB


In [155]:
res['财务风险能力']=df_risk

# 运营能力指标

In [153]:
df_operation_bpk= pd.read_excel(data_path+r'上市信息财务信息运营能力指标.xlsx')
df_operation= df_operation_bpk.copy()
J=['应收账款周转天数(天)','存货周转天数(天)']
for arr in J:
    df_operation[arr]=pd.to_numeric(df_operation[arr],errors='coerce')
df_operation['总资产周转天数']=365/df_operation['总资产周转率(次)']
df_operation.info()
df_operation=df_operation.drop(['标题','日期','总资产周转率(次)'],axis=1)
df_operation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28887 entries, 0 to 28886
Data columns (total 7 columns):
企业编号           28887 non-null int64
标题             28887 non-null object
日期             28887 non-null object
总资产周转率(次)      28797 non-null float64
应收账款周转天数(天)    27922 non-null float64
存货周转天数(天)      27776 non-null float64
总资产周转天数        28797 non-null float64
dtypes: float64(4), int64(1), object(2)
memory usage: 1.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28887 entries, 0 to 28886
Data columns (total 4 columns):
企业编号           28887 non-null int64
应收账款周转天数(天)    27922 non-null float64
存货周转天数(天)      27776 non-null float64
总资产周转天数        28797 non-null float64
dtypes: float64(3), int64(1)
memory usage: 902.8 KB


In [156]:
res['财务风险能力']=df_operation